<a href="https://colab.research.google.com/github/majekolaitan/github-search-api/blob/main/github_search_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import csv

In [ ]:
# Parameters
token = ""
search_query = "ecommerce django"
number_of_pages = 2
number_per_page = 30
starting_page = 1
sort = 'updated'
order = 'desc'

In [ ]:
def search_repos_and_create_csv():
    # GitHub API endpoint for search
    url = "https://api.github.com/search/repositories"
    output_file = search_query + '.csv'
    # Parameters for the API request
    params = {
        'q': search_query,
        'per_page': number_per_page,  # Number of results per page (max 100)
        'page': starting_page,    # Start from the first page
        'sort': sort,          # Sort by recently updated
        'order': order         # Order descending
    }

    # Headers for authentication
    headers = {
        'Authorization': f'token {token}'
    }

    # Initialize list to hold all repositories data
    all_repos = []

    for _ in range(number_of_pages):
        # Send the request to the GitHub API
        response = requests.get(url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_repos.extend(data['items'])

            # Increment the page parameter for the next request
            params['page'] += 1

            # Break the loop if there are no more items to fetch
            if len(data['items']) < 100:
                break
        else:
            print(f"Failed to fetch data from GitHub API. Status code: {response.status_code}")
            break

    # Open CSV file for writing
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header row
        writer.writerow(['Repo Name', 'Description', 'Stars', 'Watchers', 'Forks', 'Open Issues', 'Main Language', 'License', 'Score', 'Size', 'Created At', 'Updated At', 'Pushed At', 'Languages'])
        # Write the data rows
        for repo in all_repos:
            repo_name = repo['full_name']
            repo_description = repo['description']
            repo_stars = repo['stargazers_count']
            repo_watchers = repo['watchers_count']
            repo_forks = repo['forks_count']
            repo_issues_count = repo['open_issues_count']
            repo_main_language = repo['language']
            repo_license = repo['license']['name'] if repo['license'] else None
            repo_score = repo['score']
            repo_size = repo['size']
            repo_created_at = repo['created_at']
            repo_updated_at = repo['updated_at']
            repo_pushed_at = repo['pushed_at']


            # Fetch the languages used in the repository
            languages_url = repo['languages_url']
            languages_response = requests.get(languages_url, headers=headers)
            languages = languages_response.json()
            languages_used = ', '.join(languages.keys())

            # Write the row to the CSV file
            writer.writerow([
                repo_name,
                repo_description,
                repo_stars,
                repo_watchers,
                repo_forks,
                repo_issues_count,
                repo_main_language,
                repo_license,
                repo_score,
                repo_size,
                repo_created_at,
                repo_updated_at,
                repo_pushed_at,
                languages_used
            ])

    print(f"CSV file '{output_file}' created successfully.")




In [ ]:
# Run
search_repos_and_create_csv()

CSV file 'ecommerce django.csv' created successfully.
